In [ ]:
from huggingface_hub import list_repo_refs
list_repo_refs("mlsquare/exp-lora-ada-2", repo_type="model")

GitRefs(branches=[GitRefInfo(name='main', ref='refs/heads/main', target_commit='799ebd76788d9da7a2d720cb4f5eda481eb336fe')], converts=[], tags=[], pull_requests=None)

In [ ]:
from huggingface_hub import list_repo_commits
cm = list_repo_commits("mlsquare/exp-lora-ada-2", repo_type="model")
print(cm)

[GitCommitInfo(commit_id='799ebd76788d9da7a2d720cb4f5eda481eb336fe', authors=['saddlepoint'], created_at=datetime.datetime(2024, 2, 1, 2, 13, 12, tzinfo=datetime.timezone.utc), title='Upload model', message='', formatted_title=None, formatted_message=None), GitCommitInfo(commit_id='ed7b2a7cf57c0a7c25269f96048e62a6e55f8cdb', authors=['saddlepoint'], created_at=datetime.datetime(2024, 2, 1, 2, 13, 7, tzinfo=datetime.timezone.utc), title='initial commit', message='', formatted_title=None, formatted_message=None)]


In [ ]:
from modeling_mamba import MambaForCausalLM
from transformers import AutoTokenizer

model = MambaForCausalLM.from_pretrained('Q-bert/Mamba-130M')
tokenizer = AutoTokenizer.from_pretrained('Q-bert/Mamba-130M')

/Users/soma/opt/anaconda3/envs/shane/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# model merging tutorial
https://lightning.ai/lightning-ai/studios/efficient-linear-model-merging-for-llms

In [ ]:
model.config

MambaConfig {
  "_name_or_path": "Q-bert/Mamba-130M",
  "architectures": [
    "MambaForCausalLM"
  ],
  "auto_map": {
    "AutoConfig": "Q-bert/Mamba-130M--configuration_mamba.MambaConfig",
    "AutoModelForCausalLM": "Q-bert/Mamba-130M--modeling_mamba.MambaForCausalLM"
  },
  "bias": false,
  "conv_bias": true,
  "d_conv": 4,
  "d_inner": 1536,
  "d_model": 768,
  "d_state": 16,
  "dt_rank": 48,
  "expand": 2,
  "initializer_range": 0.02,
  "model_type": "mamba",
  "n_layer": 24,
  "pad_vocab_size_multiple": 8,
  "torch_dtype": "float32",
  "transformers_version": "4.37.1",
  "vocab_size": 50280
}

In [ ]:
model.state_dict().keys()

odict_keys(['model.embedding.weight', 'model.layers.0.A_log', 'model.layers.0.D', 'model.layers.0.in_proj.weight', 'model.layers.0.conv1d.weight', 'model.layers.0.conv1d.bias', 'model.layers.0.x_proj.weight', 'model.layers.0.dt_proj.weight', 'model.layers.0.dt_proj.bias', 'model.layers.0.out_proj.weight', 'model.layers.0.norm.weight', 'model.layers.1.A_log', 'model.layers.1.D', 'model.layers.1.in_proj.weight', 'model.layers.1.conv1d.weight', 'model.layers.1.conv1d.bias', 'model.layers.1.x_proj.weight', 'model.layers.1.dt_proj.weight', 'model.layers.1.dt_proj.bias', 'model.layers.1.out_proj.weight', 'model.layers.1.norm.weight', 'model.layers.2.A_log', 'model.layers.2.D', 'model.layers.2.in_proj.weight', 'model.layers.2.conv1d.weight', 'model.layers.2.conv1d.bias', 'model.layers.2.x_proj.weight', 'model.layers.2.dt_proj.weight', 'model.layers.2.dt_proj.bias', 'model.layers.2.out_proj.weight', 'model.layers.2.norm.weight', 'model.layers.3.A_log', 'model.layers.3.D', 'model.layers.3.in_pr

In [ ]:
model.eval()

MambaForCausalLM(
  (model): MambaModel(
    (embedding): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x MambaBlock(
        (in_proj): Linear(in_features=768, out_features=3072, bias=False)
        (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
        (x_proj): Linear(in_features=1536, out_features=80, bias=False)
        (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
        (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        (norm): MambaRMSNorm()
      )
    )
    (norm_f): MambaRMSNorm()
  )
  (lm_head): Linear(in_features=768, out_features=50280, bias=False)
)

In [ ]:
model.base_model.state_dict().keys()

odict_keys(['embedding.weight', 'layers.0.A_log', 'layers.0.D', 'layers.0.in_proj.weight', 'layers.0.conv1d.weight', 'layers.0.conv1d.bias', 'layers.0.x_proj.weight', 'layers.0.dt_proj.weight', 'layers.0.dt_proj.bias', 'layers.0.out_proj.weight', 'layers.0.norm.weight', 'layers.1.A_log', 'layers.1.D', 'layers.1.in_proj.weight', 'layers.1.conv1d.weight', 'layers.1.conv1d.bias', 'layers.1.x_proj.weight', 'layers.1.dt_proj.weight', 'layers.1.dt_proj.bias', 'layers.1.out_proj.weight', 'layers.1.norm.weight', 'layers.2.A_log', 'layers.2.D', 'layers.2.in_proj.weight', 'layers.2.conv1d.weight', 'layers.2.conv1d.bias', 'layers.2.x_proj.weight', 'layers.2.dt_proj.weight', 'layers.2.dt_proj.bias', 'layers.2.out_proj.weight', 'layers.2.norm.weight', 'layers.3.A_log', 'layers.3.D', 'layers.3.in_proj.weight', 'layers.3.conv1d.weight', 'layers.3.conv1d.bias', 'layers.3.x_proj.weight', 'layers.3.dt_proj.weight', 'layers.3.dt_proj.bias', 'layers.3.out_proj.weight', 'layers.3.norm.weight', 'layers.4.A_